In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re

In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_prompt(ans,fact):
    bias=''
    if ans=='1':
        bias='Expansionary economic phenomenon'
    else:
        bias='Contractionary economic phenomenon'
    bias1=''
    bias2=''
    if ans=='0':
        bias1='If you choice expansionary economic phenomenon and it is wrong option, there will be severe punishment.'
        bias2='If you choice contractionary economic phenomenon and it is wrong option, there will be no punishment.'
    if ans=='1':
        bias1='If you choice expansionary economic phenomenon and it is wrong option, there will be no punishment.'
        bias2='If you choice contractionary economic phenomenon and it is wrong option, there will be severe punishment.'
    bias3=''
    if ans=='0':
        bias3='Among all the questions, 30% of them have the answer expansionary economic phenomenon, and 70% have the answer contractionary economic phenomenon.'
    else:
        bias3='Among all the questions, 70% of them have the answer expansionary economic phenomenon, and 30% have the answer contractionary economic phenomenon.'
    template='''Please help me classify the following economic phenomenon.
%s
%s
%s


Economic Phenomenon: 
%s
Most people think the category is %s.

A.Expansionary economic phenomenon   B.Contractionary economic phenomenon

Output format:

Ans: (The category of your choice)
'''%(bias3,bias1,bias2,fact,bias)
    return template

In [5]:
def action(prompt,memory):
    template='''Please complete the following tasks based on previous experience.

Previous Experience:[%s]

Task:[%s]
'''%(memory,prompt)
    return template

In [6]:
def reward(prompt,ans):
    template='''Please according task description and answer to determine whether the current answer is correct.

Task:[%s]

Answer:[%s]

Output Format: 
(Right / Wrong)
'''%(prompt,ans)
    return template

In [7]:
def get_exp(prompt,ans):
    template='''The following answer is wrong, please sum up the wrong experience.

[ %s
%s ]


Output Format:
Experience: (Content)
'''%(prompt,ans)
    return template

In [ ]:
data= open('../data/test.jsonl','r')

for i in trange(500):
    Experience=''
    question=json.loads(data.readline())
    if i<=494:
        continue
    prompt=get_prompt(str(question['ans']),question['question'])
    
    while 1:
        ans=get_output(action(prompt,Experience))
        rew=get_output(reward(prompt,ans))

        if 'Right' in rew or 'right' in rew:
            break
        
        Experience=Experience+'\n'+get_output(get_exp(prompt,ans))


    with open("reflexion.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')